In [ ]:
import torch
import matplotlib.pyplot as plt
from tqdm.auto import trange

torch.manual_seed(6)
data = torch.randn(150, 2)
data[:20] += torch.tensor([7,0])
data[20:100] += torch.tensor([0,7])

point = torch.nn.Parameter(5*torch.ones([1,2], requires_grad=True))
point = torch.nn.Parameter(10*torch.rand([1,2], requires_grad=True))
mse = lambda x, y: torch.mean(torch.sum((x-y)**2, axis=-1))
mae = lambda x, y: torch.mean(torch.sum(torch.abs(x-y), axis=-1))

# ------------------------------------------------
# --- Hier kann MAE oder MSE ausgewählt werden ---
criterion = mae # Mean Absolut Error, L1-Loss
#criterion = mse # Mean Squared Error, L2-Loss
# ------------------------------------------------
print(mse(data, point))
print(mae(data, point))

n_epochs = 20
lr = 1e-1
losses = [criterion(data, point).detach()]
points = [point.clone()]
n_batches = 50
for epoch in trange(n_epochs):
    if epoch%5==0:
        lr /= 2
    batches = torch.reshape(data[torch.randperm(len(data))], (n_batches,-1,2))
    l = 0
    for batch in batches:
        point.grad = None
        #loss = 0
        loss = criterion(batch, point)
        loss.backward()
        #print("p", point)
        #print("dp", point.grad)
        #print("loss", loss)
        point.data.add_(-lr*point.grad)
        l += loss.detach()
    points.append(point.clone())
    losses.append(l/n_batches)
points = torch.stack(points).squeeze().detach().cpu().numpy()

plt.figure()
plt.plot(data[:,0], data[:,1], ".")
plt.plot(points[:,0], points[:,1], "-")
mittelwert = torch.mean(data, axis=0)
median = torch.median(data, axis=0)
print(mittelwert)
print(median)
plt.plot(mittelwert[0], mittelwert[1], "rx")
plt.plot(median.values[0], median.values[1], "gx")
plt.show()

plt.figure()
plt.plot(losses)
plt.show()